In [22]:
# Imports
import yaml
from IPython.display import Markdown, display
import urllib
import json
import re

# Using the Bioportal API to retrieve the mappings of terms in the eNM ontology

# Motivation
The eNanoMapper imports terms from several ontologies. For example, CHEBI and NPO both contain the class `chemical substance`, but eNanoMapper only imports the one from CHEBI [CHEBI_59999](http://purl.obolibrary.org/obo/CHEBI_59999) and not [NPO_1973](http://purl.bioontology.org/ontology/npo#NPO_1973). How do these homonymous classes map to each other?

# Approach
Using the programmatic access to Bioportal, it is possible to retrieve the mapping data originating from using LOOM. This seems to be the only way to have access to this ontology mapping tool nowadays.
The mappings will be checked to relevant, upper(ish)-level classes, among all eNanoMapper ontology dependences.

In [4]:
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)
    dependencies = config["dependencies"]
    display(Markdown("**Dependencies** \n\n" + ", ".join([i for i in dependencies])))
    API_KEY = config["api_key"]


**Dependencies** 

aopo, bao, bfo, bto, ccont, CHEBI, cheminf, chmo, cito, clo, efo, envo, fabio, go, iao, ncit, npo, oae, obcs, obi, pato, ro, sio, uberon, uo

Setting up a function to use the API ([documentation](https://data.bioontology.org/documentation#Mapping)):

In [23]:
def mappings_lookup(dependencies, iri):
    print(f"IRI: {iri}")
    iri = urllib.parse.quote(iri, safe='')
    for dependency in dependencies:
        rq = f"https://data.bioontology.org/ontologies/{dependency}/classes/{iri}/mappings"
        opener = urllib.request.build_opener()
        opener.addheaders = [('Authorization', 'apikey token=' + API_KEY)]
        try:
            r = json.loads(opener.open(rq).read())
        except Exception as e:
            pass
        else:
            print(f"Dependency: {dependency} has mappings for this IRI:")
            # Retrieve content

mappings_lookup(dependencies, "http://purl.obolibrary.org/obo/CHEBI_59999")

IRI: http://purl.obolibrary.org/obo/CHEBI_59999
Dependency: CHEBI has mappings for this IRI:
